## SRJ - Final state

## Alpha|s1> + Beta(G1|HF> + G2|HF> + G3|HF>) where theta in s2 states are fixed

In [1]:
# Preparing the term1:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []
theta = np.pi/4
operator_check = []  # To keep track of the highest operator values
alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation
beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        #print('HF stat:', hf_state)
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                #print('The ash excitation now is', ash_excitation[i])
                #print('Wires1 Invert removed are ', list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)) )
                #print('Wires2 Invert removed are',list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def s1state(hf_state, ashs, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        #print('Params in s1state are', params)
        #print('Ashs in s1state are', ashs)
        for i, excitations in enumerate(ashs):
            if len(ashs[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ashs[i][0], ashs[i][1] + 1)), wires2=list(range(ashs[i][2], ashs[i][3] + 1)))
            elif len(ashs[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ashs[i][0], ashs[i][1] + 1)))
        return qml.state()
    


    @qml.qnode(dev1)
    def s2state(hf_state, ashs, params2):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i in range(len(latest_ash)):
            #print('The value of i', i)
            if len(latest_ash[i]) == 4:
                #print('The latest ash excitation in s2 state is', latest_ash[i])
                #print('params2 is', params2[i])
                qml.DoubleExcitation(params2[i], wires = latest_ash[i]) 
            elif len(latest_ash[i]) == 2:
                qml.SingleExcitation(params2[i], wires = latest_ash[i])
        return qml.state()
    
    #@qml.qnode(dev1)
    def s2state_combined(hf_state, ashs, theta):
        """Create beta*(G1|HF⟩ + G2|HF⟩)"""
        super_states = []
        for i in range(len(ashs)):
            #params_copy = [0.0] * len(ashs)
            #params_copy[i] = params2[i]  # Only activate one excitation at a time

            # Create a separate mini-circuit for each excitation
            @qml.qnode(dev1)
            def tmp_qnode():
                [qml.PauliX(w) for w in np.nonzero(hf_state)[0]]
                #print('Excitations going to prepare s2 combined', ashs[i])
                if len(ashs[i]) == 4:
                    qml.DoubleExcitation(theta, wires=ashs[i])
                elif len(ashs[i]) == 2:
                    qml.SingleExcitation(theta, wires=ashs[i])
                return qml.state()
            
            state_i = tmp_qnode()
            super_states.append(state_i)

        summed_state = np.sum(super_states, axis=0)
        summed_state /= np.linalg.norm(summed_state)
        return summed_state

    
    dev2 = qml.device("lightning.qubit", wires=qubits, shots = 1000)
    @qml.qnode(dev2)
    def measure(state):
        qml.StatePrep(state, wires=range(qubits))
        return qml.counts()
    
    def global_cost(params, alpha, beta):
        s1o = alpha * s1state(hf_state, ashs, params)
        s2o = beta * s2state_combined(hf_state, ashs, theta)
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))
        s1oc = qml.math.conj(s1o)
        s2oc = qml.math.conj(s2o)
        # Hamiltonian expectation calculations...
        term1 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s1o)))  # <S1|H|S1>
        #term2 = np.dot(s2oc, Ham_matrix @ s2o).real  # <S2|H|S2>
        term2 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s2o))) # <S2|H|S2>
        #term3 = np.dot(s1oc, Ham_matrix @ s2o).real  # <S1|H|S2>
        term3 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s2o)))  # <S1|H|S2>
        #term4 = np.dot(s2oc, Ham_matrix @ s1o).real  # <S2|H|S1>
        term4 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s1o)))  # <S2|H|S1>
        numerator = term1 + term2 + term3 + term4
        #D1 = np.dot(s1oc, s1o).real
        D1 = qml.math.real(qml.math.dot(s1oc, s1o))  # <S1|S1>
        #D2 = np.dot(s2oc, s2o).real
        D2 = qml.math.real(qml.math.dot(s2oc, s2o))  # <S2|S2>
        #D3 = np.dot(s1oc, s2o).real
        D3 = qml.math.real(qml.math.dot(s1oc, s2o))  # <S1|S2>
        #D4 = np.dot(s2oc, s1o).real
        D4 = qml.math.real(qml.math.dot(s2oc, s1o))  # <S2|H|S1>
        denominator = D1 + D2 + D3 + D4
        return numerator / denominator

    def global_cost_flat(x):
        params = x[:len(ashs)]  # 0,1
        #params2 = x[len(ashs):len(ashs) + 1] #2
        alpha = x[len(ashs)] #3
        beta = x[len(ashs)+1]  #4
        return global_cost(params, alpha, beta)

    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

    null_state = np.zeros(qubits,int)
    #print('Null state is', null_state)
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            if i not in operator_check:
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        operator_check.append(max_operator)
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        ashs = ash_excitation
        print('Len of ashs:', len(ashs))
        alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)
        print('Exc. going to prepare state1',ashs)
        params = np.zeros(len(ashs), requires_grad=True)    
        #params = np.append(params, 0.0)  #Parameters initialization



        s1ostate = alpha * s1state(hf_state, ashs, params)
        print(qml.draw(s1state, max_length=100)(hf_state,ashs,params))

        #s2 state preparation
        print('Preparing s2 state with ashs:', ashs)
        beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)

        
        s2ostate = beta * s2state_combined(hf_state, ashs, theta)
        print(qml.draw(s2state_combined, max_length=100)(hf_state,ashs,theta))
        x0 = np.concatenate([params, np.array([alpha]), np.array([beta])])

        print(x0)




        result = minimize(global_cost_flat, x0, method='BFGS')
        print(result)
        energies.append(result.fun)  # Appending the energy to the energies list
        params = result.x[:len(ashs)]  # Extracting the first part of the result as params
        #params2 = result.x[len(ashs):len(ashs) + 1]  # Extracting the second part of the result as params2
        print('Params1 after optimization:', params)
        alpha = result.x[len(ashs)]  # Extracting the alpha parameter
        beta = result.x[len(ashs) + 1]  # Extracting the beta parameter
        print('Alpha after optimization:', alpha)
        print('Beta after optimization:', beta)

        final_state = alpha * s1state(hf_state, ashs, params) + beta * s2state_combined(hf_state, ashs, theta)
        final_state = final_state / np.linalg.norm(final_state)

        #final_state = s1state(hf_state, ashs, params)
        print("Norm after manual normalization:", np.linalg.norm(final_state)) 
        print('counts after the final state with optimization is', measure(final_state))
        states.append(final_state)  # Appending the new state to the states list
        
    return params, ash_excitation, qubits, H, energies, old_grad




symbols  = [ 'H', 'H', 'H', 'H']
print('H4-3A-GS-BFGS-sto6g')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])



#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 4
active_orbitals = 4 #Thinkng it is spatial 


params, ash_excitation, qubits, H,energies, old_grad = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=25) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
HF state is -1.3325688880668933
Total excitations are 26
The adapt iteration now is 0
The highest operator value is 0.3100451588582031 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
Len of ashs: 1
Exc. going to prepare state1 [[2, 3, 6, 7]]
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(0.00)─┤  State
3: ──X─├FermionicDoubleExcitation(0.00)─┤  State
6: ────├FermionicDoubleExcitation(0.00)─┤  State
7: ────╰FermionicDoubleExcitation(0.00)─┤  State
Preparing s2 state with ashs: [[2, 3, 6, 7]]

[0. +0.j  0.6+0.8j 0.6+0.8j]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.4300836708312177
        x: [3.22407893+0.j  0.33711562+0.8j 1.74123946+0.8j]
      nit: 14
      jac: [-1.639e-07 -1.639e-06  3.129e-07]
 hess_inv: [[ 6.006e+02 -4.478e+00  1.983e+02]
   

In [2]:
energies

[-1.4300836708312177,
 -1.569460382907646,
 -1.7440626582296175,
 -1.8660667756649474,
 -1.865562062247596,
 -1.8663977847643638,
 -1.86675372909011,
 -1.8847075928739458,
 -1.8846952659783278,
 -1.8848634162091664,
 -1.8847982523647195,
 -1.8849221392235702,
 -1.8849218655977298,
 -1.8849209854956515,
 -1.8849209617451081,
 -1.8849209412902004,
 -1.884920923423662,
 -1.8849209071106183,
 -1.8849208856016182,
 -1.884920872466914,
 -1.884920859673833,
 -1.8849208504398727,
 -1.8849208306127594,
 -1.8849208212744535,
 -1.884920825724639]

In [3]:
old_grad

[0.3100451588582031,
 0.3250591214097044,
 0.42559362972620557,
 0.3089790551359852,
 0.08920458777623344,
 0.10166295344580219,
 0.13980941049621365,
 0.1454141387285073,
 0.033742405789960836,
 0.03470533432805096,
 0.0021592784314326896,
 0.012996052986520098,
 0.01230240609657167,
 0.007356562956429606,
 1.0633462371586084e-16,
 0.005220475396232573,
 0.005970526343274208,
 0.001407763571020783,
 0.0036241458304085788,
 0.0010973282554906435,
 0.003680970482578247,
 0.005975393378971383,
 0.003280377182702089,
 0.005612299463911468,
 0.0018235241643706915]

## Using repetitive operators

In [1]:
# Preparing the term1:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []
theta = np.pi/4
operator_check = []  # To keep track of the highest operator values
alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation
beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        #print('HF stat:', hf_state)
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                #print('The ash excitation now is', ash_excitation[i])
                #print('Wires1 Invert removed are ', list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)) )
                #print('Wires2 Invert removed are',list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def s1state(hf_state, ashs, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        #print('Params in s1state are', params)
        #print('Ashs in s1state are', ashs)
        for i, excitations in enumerate(ashs):
            if len(ashs[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ashs[i][0], ashs[i][1] + 1)), wires2=list(range(ashs[i][2], ashs[i][3] + 1)))
            elif len(ashs[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ashs[i][0], ashs[i][1] + 1)))
        return qml.state()
    



    
    #@qml.qnode(dev1)
    def s2state_combined(hf_state, ashs, theta):
        """Create beta*(G1|HF⟩ + G2|HF⟩)"""
        super_states = []
        for i in range(len(ashs)):
            #params_copy = [0.0] * len(ashs)
            #params_copy[i] = params2[i]  # Only activate one excitation at a time

            # Create a separate mini-circuit for each excitation
            @qml.qnode(dev1)
            def tmp_qnode():
                [qml.PauliX(w) for w in np.nonzero(hf_state)[0]]
                #print('Excitations going to prepare s2 combined', ashs[i])
                if len(ashs[i]) == 4:
                    qml.DoubleExcitation(theta, wires=ashs[i])
                elif len(ashs[i]) == 2:
                    qml.SingleExcitation(theta, wires=ashs[i])
                return qml.state()
            
            state_i = tmp_qnode()
            super_states.append(state_i)

        summed_state = np.sum(super_states, axis=0)
        summed_state /= np.linalg.norm(summed_state)
        return summed_state

    
    dev2 = qml.device("lightning.qubit", wires=qubits, shots = 1000)
    @qml.qnode(dev2)
    def measure(state):
        qml.StatePrep(state, wires=range(qubits))
        return qml.counts()
    
    def global_cost(params, alpha, beta):
        s1o = alpha * s1state(hf_state, ashs, params)
        s2o = beta * s2state_combined(hf_state, ashs, theta)
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))
        s1oc = qml.math.conj(s1o)
        s2oc = qml.math.conj(s2o)
        # Hamiltonian expectation calculations...
        term1 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s1o)))  # <S1|H|S1>
        #term2 = np.dot(s2oc, Ham_matrix @ s2o).real  # <S2|H|S2>
        term2 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s2o))) # <S2|H|S2>
        #term3 = np.dot(s1oc, Ham_matrix @ s2o).real  # <S1|H|S2>
        term3 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s2o)))  # <S1|H|S2>
        #term4 = np.dot(s2oc, Ham_matrix @ s1o).real  # <S2|H|S1>
        term4 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s1o)))  # <S2|H|S1>
        numerator = term1 + term2 + term3 + term4
        #D1 = np.dot(s1oc, s1o).real
        D1 = qml.math.real(qml.math.dot(s1oc, s1o))  # <S1|S1>
        #D2 = np.dot(s2oc, s2o).real
        D2 = qml.math.real(qml.math.dot(s2oc, s2o))  # <S2|S2>
        #D3 = np.dot(s1oc, s2o).real
        D3 = qml.math.real(qml.math.dot(s1oc, s2o))  # <S1|S2>
        #D4 = np.dot(s2oc, s1o).real
        D4 = qml.math.real(qml.math.dot(s2oc, s1o))  # <S2|H|S1>
        denominator = D1 + D2 + D3 + D4
        return numerator / denominator

    def global_cost_flat(x):
        params = x[:len(ashs)]  # 0,1
        #params2 = x[len(ashs):len(ashs) + 1] #2
        alpha = x[len(ashs)] #3
        beta = x[len(ashs)+1]  #4
        return global_cost(params, alpha, beta)

    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

    null_state = np.zeros(qubits,int)
    #print('Null state is', null_state)
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            print(f'The expectation value of {i} is', current_value)
            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        operator_check.append(max_operator)
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        ashs = ash_excitation
        print('Len of ashs:', len(ashs))
        alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)
        print('Exc. going to prepare state1',ashs)
        params = np.zeros(len(ashs), requires_grad=True)    
        #params = np.append(params, 0.0)  #Parameters initialization



        s1ostate = alpha * s1state(hf_state, ashs, params)
        print(qml.draw(s1state, max_length=100)(hf_state,ashs,params))

        #s2 state preparation
        print('Preparing s2 state with ashs:', ashs)
        beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)

        
        s2ostate = beta * s2state_combined(hf_state, ashs, theta)
        #print(qml.draw(s2state_combined, max_length=100)(hf_state,ashs,theta))
        x0 = np.concatenate([params, np.array([alpha]), np.array([beta])])

        #print(x0)




        result = minimize(global_cost_flat, x0, method='BFGS')
        print(result)
        energies.append(result.fun)  # Appending the energy to the energies list
        params = result.x[:len(ashs)]  # Extracting the first part of the result as params
        #params2 = result.x[len(ashs):len(ashs) + 1]  # Extracting the second part of the result as params2
        print('Params1 after optimization:', params)
        alpha = result.x[len(ashs)]  # Extracting the alpha parameter
        beta = result.x[len(ashs) + 1]  # Extracting the beta parameter
        print('Alpha after optimization:', alpha)
        print('Beta after optimization:', beta)

        final_state = alpha * s1state(hf_state, ashs, params) + beta * s2state_combined(hf_state, ashs, theta)
        final_state = final_state / np.linalg.norm(final_state)

        #final_state = s1state(hf_state, ashs, params)
        print("Norm after manual normalization:", np.linalg.norm(final_state)) 
        print('counts after the final state with optimization is', measure(final_state))
        states.append(final_state)  # Appending the new state to the states list
        
    return params, ash_excitation, qubits, H, energies, old_grad




symbols  = [ 'H', 'H', 'H', 'H']
print('H4-3A-GS-BFGS-sto6g')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])



#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 4
active_orbitals = 4 #Thinkng it is spatial 


params, ash_excitation, qubits, H,energies, old_grad = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=25) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
HF state is -1.3325688880668933
Total excitations are 26
The adapt iteration now is 0
The expectation value of a⁺(0) a(4) is 9.258246501386846e-08
The expectation value of a⁺(0) a(6) is 0.0
The expectation value of a⁺(1) a(5) is 9.258246502774625e-08
The expectation value of a⁺(1) a(7) is 0.0
The expectation value of a⁺(2) a(4) is 0.0
The expectation value of a⁺(2) a(6) is 1.1699021076894134e-07
The expectation value of a⁺(3) a(5) is 0.0
The expectation value of a⁺(3) a(7) is 1.1699021078281913e-07
The expectation value of a⁺(0) a⁺(1) a(4) a(5) is 0.30574752590578796
The expectation value of a⁺(0) a⁺(1) a(4) a(7) is 0.0
The expectation value of a⁺(0) a⁺(1) a(5) a(6) is 0.0
The expectation value of a⁺(0) a⁺(1) a(6) a(7) is 0.28410686098080123
The expectation value of a⁺(0) a⁺(2) a(4) a(6) is 0.030544625693058606
The expectation value of a⁺(0) a⁺(3) a(4) a(5) is 0.0
The expectation value of a⁺(0) a⁺(3) a(4) a(7

## Optimizing the params2 also

In [1]:
# Preparing the term1:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []
theta = np.pi/4
operator_check = []  # To keep track of the highest operator values
alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation
beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)

    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        #print('HF stat:', hf_state)
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                #print('The ash excitation now is', ash_excitation[i])
                #print('Wires1 Invert removed are ', list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)) )
                #print('Wires2 Invert removed are',list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return

    

    
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def s1state(hf_state, ashs, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        #print('Params in s1state are', params)
        #print('Ashs in s1state are', ashs)
        for i, excitations in enumerate(ashs):
            if len(ashs[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ashs[i][0], ashs[i][1] + 1)), wires2=list(range(ashs[i][2], ashs[i][3] + 1)))
            elif len(ashs[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ashs[i][0], ashs[i][1] + 1)))
        return qml.state()
    



    
    #@qml.qnode(dev1)
    def s2state_combined(hf_state, ashs, params2):
        """Construct beta * ∑_i G_i |HF> where G_i applies the i-th excitation"""
        super_states = []

        for i in range(len(ashs)):
            excitation = ashs[i]

            # QNode that applies only the i-th excitation
            @qml.qnode(dev1)
            def tmp_qnode():
                for w in np.nonzero(hf_state)[0]:
                    qml.PauliX(w)
                
                # Apply the selected excitation
                if len(excitation) == 4:
                    qml.DoubleExcitation(params2[i], wires=excitation)
                elif len(excitation) == 2:
                    qml.SingleExcitation(params2[i], wires=excitation)
                
                return qml.state()

            state_i = tmp_qnode()
            super_states.append(state_i)

        # Sum all individual excitation states
        summed_state = qml.numpy.sum(qml.numpy.stack(super_states), axis=0)

        # Normalize the superposition state
        norm = qml.math.sqrt(qml.math.real(
            qml.math.dot(qml.math.conj(summed_state), summed_state)
        ))
        normalized_state = summed_state / norm

        return normalized_state


    
    dev2 = qml.device("lightning.qubit", wires=qubits, shots = 1000)
    @qml.qnode(dev2)
    def measure(state):
        qml.StatePrep(state, wires=range(qubits))
        return qml.counts()
    
    def global_cost(params, params2, alpha, beta):
        s1o = alpha * s1state(hf_state, ashs, params)
        s2o = beta * s2state_combined(hf_state, ashs, params2)
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))
        s1oc = qml.math.conj(s1o)
        s2oc = qml.math.conj(s2o)
        # Hamiltonian expectation calculations...
        term1 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s1o)))  # <S1|H|S1>
        #term2 = np.dot(s2oc, Ham_matrix @ s2o).real  # <S2|H|S2>
        term2 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s2o))) # <S2|H|S2>
        #term3 = np.dot(s1oc, Ham_matrix @ s2o).real  # <S1|H|S2>
        term3 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s2o)))  # <S1|H|S2>
        #term4 = np.dot(s2oc, Ham_matrix @ s1o).real  # <S2|H|S1>
        term4 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s1o)))  # <S2|H|S1>
        numerator = term1 + term2 + term3 + term4
        #D1 = np.dot(s1oc, s1o).real
        D1 = qml.math.real(qml.math.dot(s1oc, s1o))  # <S1|S1>
        #D2 = np.dot(s2oc, s2o).real
        D2 = qml.math.real(qml.math.dot(s2oc, s2o))  # <S2|S2>
        #D3 = np.dot(s1oc, s2o).real
        D3 = qml.math.real(qml.math.dot(s1oc, s2o))  # <S1|S2>
        #D4 = np.dot(s2oc, s1o).real
        D4 = qml.math.real(qml.math.dot(s2oc, s1o))  # <S2|H|S1>
        denominator = D1 + D2 + D3 + D4
        return numerator / denominator

    
    def global_cost_flat(x):
        len1 = len(ashs)
        len2 = len(ashs)
        params = x[:len1]
        params2 = x[len1:len1+len2]
        alpha   = x[len1 + len2]
        beta    = x[len1 + len2 + 1]
        return global_cost(params,params2, alpha, beta)

    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

    null_state = np.zeros(qubits,int)
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            #print(f'The expectation value of {i} is', current_value)
            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        operator_check.append(max_operator)
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        ashs = ash_excitation
        print('Len of ashs:', len(ashs))
        alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)
        print('Exc. going to prepare state1',ashs)
        params = np.zeros(len(ashs), requires_grad=True)    
        #params = np.append(params, 0.0)  #Parameters initialization



        s1ostate = alpha * s1state(hf_state, ashs, params)
        print(qml.draw(s1state, max_length=100)(hf_state,ashs,params))

        #s2 state preparation
        print('Preparing s2 state with ashs:', ashs)
        params2 = qml.numpy.array([np.pi/4] * len(ashs), requires_grad=True)
        beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)

        
        s2ostate = beta * s2state_combined(hf_state, ashs, params2)
        #print(qml.draw(s2state_combined, max_length=100)(hf_state,ashs,theta))
        x0 = np.concatenate([params, params2, np.array([alpha]), np.array([beta])])

        #print(x0)




        result = minimize(global_cost_flat, x0, method='BFGS')
        print(result)
        energies.append(result.fun)  # Appending the energy to the energies list
        len1 = len(ashs)
        len2 = len(ashs)
        params = result.x[:len1]  # Extracting the first part of the result as params
        params2 = result.x[len1:len1+len2]  # Extracting the second part of the result as params2
        print('Params1 after optimization:', params)
        print('Params2 after optimization:', params2)
        alpha = result.x[len1+len2]  # Extracting the alpha parameter
        beta = result.x[len1+len2+ 1]  # Extracting the beta parameter
        print('Alpha after optimization:', alpha)
        print('Beta after optimization:', beta)

        #final_state = alpha * s1state(hf_state, ashs, params) + beta * s2state_combined(hf_state, ashs, params2)
        final_state =   beta * s2state_combined(hf_state, ashs, params2)

        final_state = final_state / np.linalg.norm(final_state)

        #final_state = s1state(hf_state, ashs, params)
        print("Norm after manual normalization:", np.linalg.norm(final_state)) 
        print('counts after the final state with optimization is', measure(final_state))
        states.append(final_state)  # Appending the new state to the states list
        
    return params, ash_excitation, qubits, H, energies, old_grad




symbols  = [ 'H', 'H', 'H', 'H']
print('H4-3A-GS-BFGS-sto6g')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])



#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 4
active_orbitals = 4 #Thinkng it is spatial 


params, ash_excitation, qubits, H,energies, old_grad = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=15) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
HF state is -1.3325688880668933
Total excitations are 26
The adapt iteration now is 0
The highest operator value is 0.3100451588582031 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
Len of ashs: 1
Exc. going to prepare state1 [[2, 3, 6, 7]]
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(0.00)─┤  State
3: ──X─├FermionicDoubleExcitation(0.00)─┤  State
6: ────├FermionicDoubleExcitation(0.00)─┤  State
7: ────╰FermionicDoubleExcitation(0.00)─┤  State
Preparing s2 state with ashs: [[2, 3, 6, 7]]
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.4300836706621738
        x: [0.48841151+0.j  1.29888562+0.j  0.26021239+0.8j 0.92400212+0.8j]
      nit: 4
      jac: [-1.654e-06 -6.169e-06  4.172e-06 -1.192e-06]
 hess_inv: [[ 3.155e+00  2.233e+00 -1.476e+00  1.428e+

## For s2 state : ashs[:-1]

## Non - Repetitive operators

In [1]:
# Preparing the term1:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
warnings.filterwarnings("ignore")
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []
theta = np.pi/4
operator_check = []  # To keep track of the highest operator values
alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation
beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)  #Parameter for the excitation

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)

    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)




    

    
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def s1state(hf_state, ashs, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        #print('Params in s1state are', params)
        #print('Ashs in s1state are', ashs)
        for i, excitations in enumerate(ashs):
            if len(ashs[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ashs[i][0], ashs[i][1] + 1)), wires2=list(range(ashs[i][2], ashs[i][3] + 1)))
            elif len(ashs[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ashs[i][0], ashs[i][1] + 1)))
        return qml.state()
    



    
    #@qml.qnode(dev1)
    def s2state_combined(hf_state, ashs, params2):
        """Construct beta * ∑_i G_i |HF> where G_i applies the i-th excitation"""
        super_states = []

        for i in range(len(ashs)):
            excitation = ashs[i]

            # QNode that applies only the i-th excitation
            @qml.qnode(dev1)
            def tmp_qnode():
                for w in np.nonzero(hf_state)[0]:
                    qml.PauliX(w)
                
                # Apply the selected excitation
                if len(excitation) == 4:
                    qml.DoubleExcitation(params2[i], wires=excitation)
                elif len(excitation) == 2:
                    qml.SingleExcitation(params2[i], wires=excitation)
                
                return qml.state()

            state_i = tmp_qnode()
            super_states.append(state_i)

        # Sum all individual excitation states
        summed_state = qml.numpy.sum(qml.numpy.stack(super_states), axis=0)

        # Normalize the superposition state
        norm = qml.math.sqrt(qml.math.real(
            qml.math.dot(qml.math.conj(summed_state), summed_state)
        ))
        normalized_state = summed_state / norm

        return normalized_state


    
    dev2 = qml.device("lightning.qubit", wires=qubits, shots = 1000)
    @qml.qnode(dev2)
    def measure(state):
        qml.StatePrep(state, wires=range(qubits))
        return qml.counts()
    
    def global_cost(params, params2, alpha, beta):
        s1o = alpha * s1state(hf_state, ashs, params)
        if len(ashs) > 1:
            s2o = beta * s2state_combined(hf_state, ashs[:-1], params2)
        else:
            s2o = qml.numpy.zeros_like(s1ostate) 
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))
        s1oc = qml.math.conj(s1o)
        s2oc = qml.math.conj(s2o)
        # Hamiltonian expectation calculations...
        term1 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s1o)))  # <S1|H|S1>
        #term2 = np.dot(s2oc, Ham_matrix @ s2o).real  # <S2|H|S2>
        term2 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s2o))) # <S2|H|S2>
        #term3 = np.dot(s1oc, Ham_matrix @ s2o).real  # <S1|H|S2>
        term3 = qml.math.real(qml.math.dot(s1oc, qml.math.dot(Ham_matrix, s2o)))  # <S1|H|S2>
        #term4 = np.dot(s2oc, Ham_matrix @ s1o).real  # <S2|H|S1>
        term4 = qml.math.real(qml.math.dot(s2oc, qml.math.dot(Ham_matrix, s1o)))  # <S2|H|S1>
        numerator = term1 + term2 + term3 + term4
        #D1 = np.dot(s1oc, s1o).real
        D1 = qml.math.real(qml.math.dot(s1oc, s1o))  # <S1|S1>
        #D2 = np.dot(s2oc, s2o).real
        D2 = qml.math.real(qml.math.dot(s2oc, s2o))  # <S2|S2>
        #D3 = np.dot(s1oc, s2o).real
        D3 = qml.math.real(qml.math.dot(s1oc, s2o))  # <S1|S2>
        #D4 = np.dot(s2oc, s1o).real
        D4 = qml.math.real(qml.math.dot(s2oc, s1o))  # <S2|H|S1>
        denominator = D1 + D2 + D3 + D4
        return numerator / denominator

    
    def global_cost_flat(x):
        len1 = len(ashs)
        len2 = len(ashs[:-1])
        params = x[:len1]
        params2 = x[len1:len1+len2]
        alpha   = x[len1 + len2]
        beta    = x[len1 + len2 + 1]
        return global_cost(params,params2, alpha, beta)

    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

    null_state = np.zeros(qubits,int)
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            if i not in operator_check:
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
                #print(f'The expectation value of {i} is', current_value)
                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        operator_check.append(max_operator)
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        ashs = ash_excitation
        print('Len of ashs:', len(ashs))
        alpha = qml.numpy.array(0.6 + 0.8j, requires_grad=True)
        print('Exc. going to prepare state1',ashs)
        params = np.zeros(len(ashs), requires_grad=True)    
        #params = np.append(params, 0.0)  #Parameters initialization



        s1ostate = alpha * s1state(hf_state, ashs, params)
        print(qml.draw(s1state, max_length=100)(hf_state,ashs,params))

        #s2 state preparation
        if len(ashs) > 1: 
            print('Preparing s2 state with ashs:', ashs[:-1])
            params2 = qml.numpy.array([np.pi/4] * len(ashs[:-1]), requires_grad=True)
            beta = qml.numpy.array(0.6 + 0.8j, requires_grad=True)
            s2ostate = beta * s2state_combined(hf_state, ashs[:-1], params2)
        else:
            s2ostate = 0 * s1ostate
            params2 = qml.numpy.array([], requires_grad=True)
            beta = 0.0
        #print(qml.draw(s2state_combined, max_length=100)(hf_state,ashs,theta))
        x0 = np.concatenate([params, params2, np.array([alpha]), np.array([beta])])

        #print(x0)




        result = minimize(global_cost_flat, x0, method='BFGS')
        print(result)
        energies.append(result.fun)  # Appending the energy to the energies list
        len1 = len(ashs)
        len2 = len(ashs[:-1])
        params = result.x[:len1]  # Extracting the first part of the result as params
        params2 = result.x[len1:len1+len2]  # Extracting the second part of the result as params2
        print('Params1 after optimization:', params)
        print('Params2 after optimization:', params2)
        alpha = result.x[len1+len2]  # Extracting the alpha parameter
        beta = result.x[len1+len2+ 1]  # Extracting the beta parameter
        print('Alpha after optimization:', alpha)
        print('Beta after optimization:', beta)

        #final_state = alpha * s1state(hf_state, ashs, params) + beta * s2state_combined(hf_state, ashs, params2)
        final_state =    s1state(hf_state, ashs, params)

        final_state = final_state / np.linalg.norm(final_state)

        #final_state = s1state(hf_state, ashs, params)
        print("Norm after manual normalization:", np.linalg.norm(final_state)) 
        print('counts after the final state with optimization is', measure(final_state))
        states.append(final_state)  # Appending the new state to the states list
        print('\n\n')
    return params, ash_excitation, qubits, H, energies, old_grad




symbols  = [ 'H', 'H', 'H', 'H']
print('H4-3A-GS-BFGS-sto6g')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 3.0*r_bohr], [0.0,0.0,6.0*r_bohr],[0.0, 0.0, 9.0*r_bohr]])



#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 4
active_orbitals = 4 #Thinkng it is spatial 


params, ash_excitation, qubits, H,energies, old_grad = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=15) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


H4-3A-GS-BFGS-sto6g
Using active space, check if you change the H accordingly
HF state is -1.3325688880668944
Total excitations are 26
The adapt iteration now is 0
The highest operator value is 0.3100451588582133 for operator a⁺(2) a⁺(3) a(6) a(7)
Highest gradient excitation is [2, 3, 6, 7]
Len of ashs: 1
Exc. going to prepare state1 [[2, 3, 6, 7]]
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(0.00)─┤  State
3: ──X─├FermionicDoubleExcitation(0.00)─┤  State
6: ────├FermionicDoubleExcitation(0.00)─┤  State
7: ────╰FermionicDoubleExcitation(0.00)─┤  State
  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -1.4300836708388682
        x: [1.12299682+0.j  0.6000001 +0.8j 0.        +0.j ]
      nit: 4
      jac: [ 7.302e-07  0.000e+00  0.000e+00]
 hess_inv: [[ 5.829e+00  4.483e-07  0.000e+00]
            [ 4.483e-07  1.000e+00  0.000e+00]
            [ 0.000e+00  0.000e+

In [4]:
FCI = -1.8851249037689082
energy = -1.8851239471284795

energy - FCI


9.566404286509567e-07